### Exercícios práticos 04 - Syanne Tavares

#### Resolução de sistemas lineares

- Podem ter 3 situações

1. Solução única -> retas concorrentes -> se tocam em 1 único ponto
2. Infinitas soluções -> retas coicidentes - pode ser qualquer tipo de valor
3. Sem solução -> Reta paralelas 

- 2 formas de resolver 
1. Métodos Diretos 
2. Métodos iterativos


## Exercício 01 - 

Implemente todas as técnicas de solução de sistemas lineares vistas em aula e faça uma comparação entre elas para as seguintes situações

# Métodos Diretos
## Fornecem soluções exatas com um número finito de operações. Podem ocorrer erros de arredondamento


# 1. Eliminação de Gauss
Resumidamente -> transforma o sistema em um sistema equivalente do tipo triangular superior e resolve a partir disso

In [2]:
import numpy as np
import pandas as pd

def pivoteamentoParcial(A):

  return A
  
# Resolve o sistema triangular superior.
def SistemaTriangularSuperior(Ax,b):
  """Função que resolve um sistema linear do tipo  𝐴𝑥 = b """
  n = len(b) #tamanho do vetor b
  x= [b[i]/Ax[i][i] if i == 0 else None for i in range(0,len(b))]

  x[n-1] = b[n-1]/Ax[n-1][n-1]

  for i in list(range(n-1,0,-1)):
    s = 0
    for j in list(range(i+1,n+1)):
      s = s + Ax[i-1][j-1]*x[j-1]

    x[i-1] = (b[i-1]-s)/(Ax[i-1][i-1])

  return x


def eliminacao(A,b):
  n = len(b)
  x = [0]*n

  # Calculo dos pivos.
  for k in list(range(1,n)):
    # PIVOTEAMENTO PARCIAL
    for i in range(k+1, len(A)+1):
      if (abs(A[i-1][k-1]) > abs(A[k-1][k-1])):
        [A[k-1], A[i-1]] = [A[i-1], A[k-1]]
        [b[k-1], b[i-1]] = [b[i-1], b[k-1]]
    # Calculo dos multiplicadores.
    for i in list(range(k+1,n+1)):
      m = A[i-1][k-1]/A[k-1][k-1]
      A[i-1][k-1] = 0
      # Atualizar demais valores da linha
      for j in list(range(k+1,n+1)):
        A[i-1][j-1] = A[i-1][j-1]-m*A[k-1][j-1]
      b[i-1] = b[i-1] - m*b[k-1]

  return A,b

def EliminacaoDeGauss(Ax,b):
  A = pivoteamentoParcial(Ax)
  Ax2,b2 = eliminacao(A,b)
 
  print("depois da eliminação\n Ax =  ",Ax2,"\nb = ",b2)
  x = SistemaTriangularSuperior(Ax2,b2)
  return(x)


In [3]:
Ax = [[3, 2, 4],
      [1, 1, 2],
      [4, 3,-2]]
b = [1, 2, 3]

print("X: ",EliminacaoDeGauss(Ax,b))

depois da eliminação
 Ax =   [[4, 3, -2], [0, 0.25, 2.5], [0, 0, 8.0]] 
b =  [3, 1.25, 0.0]
X:  [-3.0, 5.0, 0.0]


# Fatoração LU com pivoteamento parcial

#### O processo de fatoração consistem em decompor a matriz A em um produto de dois ou mais fatores
▪ De forma que seja mais fácil resolver o sistema em etapas

In [4]:
def formata_matriz(M):
  m = len(M) # número de linhas
  n = len(M[0]) # número de colunas
  s = ""
  for i in range(m):
      for j in range(n):
          s += "%9.3f " % M[i][j]
      s +=  "\n"
  return s

# Resolve o sistema triangular inferior.
def SistemaTriangularInferior(L,b):
  n = len(b)
  
  y= [b[i]/L[i][i] if i == 0 else None for i in range(0,len(b))]

  for i in list(range(1,n+1,1)):
    s = 0
    for j in list(range(1,i,1)):
      s = s + L[i-1][j-1]*y[j-1]

    y[i-1] = b[i-1] - s

  return y

def LU_PivParc(A,b):
  n = len(A)
  p=[0]*n
  

  ###### pivoteamento parcial 
  p = list(range(1,n+1)) #"matriz permutação"

  for k in range(1,n):
    pv = abs(A[k-1][k-1])
    Lpivo = k

    for i in range(k+1,n+1):
        if abs(A[i-1][k-1])> pv :
          pv = abs(A[i-1][k-1])
          Lpivo = i
  
    if pv == 0: 
      print("Matriz Singular")
      break

    if Lpivo != k : # se for diferente então tem que realizar a permutação
      [p[k-1],p[Lpivo-1]] = [p[Lpivo-1],p[k-1]] # recurso guanabaristico
    
      for j in range(1,n+1):
          [A[k-1][j-1],A[Lpivo-1][j-1]] = [A[Lpivo-1][j-1],A[k-1][j-1]] 

  # resolução dos sistemas triangulares
  # Calculo dos pivos.
  for k in list(range(1,n,1)):
    # Calculo dos multiplicadores.
    for i in list(range(k+1,n+1,1)):
      m = A[i-1][k-1]/A[k-1][k-1]
      A[i-1][k-1] = m
      # Atualizar demais valores da linha
      for j in list(range(k+1,n+1,1)):
        A[i-1][j-1] = A[i-1][j-1]-m*A[k-1][j-1]
  ### multiplicação da b pela "matriz permutação"
  c=[None for _ in range(0,len(b))]
  for i in range(1,n+1):
    Lpivo = p[i-1]
    c[i-1]=b[Lpivo-1]

  # resolução dos sistemas 
  y = SistemaTriangularInferior(A,c)
  x = SistemaTriangularSuperior(A,y)

  return x

In [6]:
# Teste com dados.
Ai = [[3, -4, 1],
      [1, 2, 2],
      [4, 0, -3]]

bi = [9, 3, -2]

print("Ai = \n",formata_matriz(Ai))
#Obtendo os fatores L e U.
print("X =")
print(LU_PivParc(Ai,bi))

Ai = 
     3.000    -4.000     1.000 
    1.000     2.000     2.000 
    4.000     0.000    -3.000 

X =
[1.0, -1.0, 2.0]


In [124]:
print(np.eye(3))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## Métodos iterativos

### 3. Gauss-Jacobi

In [7]:
import math
def TesteConvergencia(MatrizA):
    Alfas=list()

    for i in range(len(MatrizA)):
        s=0
        for j in range(len(MatrizA)):
            if j!=i:
                s+=MatrizA[i][j]

        Alfas.append(s/MatrizA[i][i])
    # print(Alfas)
    # print(max(Alfas))

    if max(Alfas) < 1:
        return True
    else:
        return False
def criterioParada(x,xk,precisao):
    soma = 0
    zip_object = zip(x, xk) # une as informações de duas variáveis
    for list1_i, list2_i in zip_object:
        soma = soma + math.fabs(list1_i-list2_i)

    if (soma < precisao):
        return True
    else:
        return False   
            
def GaussJacobi(MatrizA,b,max_iteracoes,precisao):
    n= len(b) #tamanho de do vetor base

    X=b.copy() # copiando b pra x como chute inicial
    solucao = True
    # gerar chute inicial
    for i in list(range(1,n+1)): # percorre cada elemento de x de 1 até n
        # dividir pelo elemento da diagonal principal
        if math.fabs(MatrizA[i-1][i-1])> 0 :# verifica se é possível realizar divisão se a diagonal é diferente de zero
            X[i-1] = b[i-1]/MatrizA[i-1][i-1]
        else:
            solucao = False # não se pode calcular o chute inicial
            break
    if solucao and TesteConvergencia(MatrizA):
        print("Iteração 0")
        print("x = ",X)

    xk = X.copy()

    iter    = 0
 
    while (iter < max_iteracoes): # início das iterações
      iter = iter + 1
      for i in list(range(1,n+1)):
        s = 0

        #realiza o somatório
        for j in list(range(1,n+1)):
          if ((i-1) != (j-1)):
            s = s + MatrizA[i-1][j-1]*X[j-1] # somando os termos que não são da diagonal

        xk[i-1] = (1/MatrizA[i-1][i-1])*(b[i-1]-s) # equivale a 1/a(ii)*(bi - somatório de aij*xj)
     
      print("Iteração: ",iter)
      print("xk = ",xk)

      #verifica o critério de parada ->  O processo é repetido até ue x^k e x^k-1 eestejam suficientemente próximos
      if criterioParada(X,xk,precisao):
        X = xk.copy()
        break    
      X = xk.copy()
     
   
    return X







In [9]:
A = [[10, 2,  1],
     [ 1, 5,  1],
     [ 2, 3, 10]]

b = [7, -8, 6]

x = GaussJacobi(A,b,10,0.01)
print("x = ",x)


Iteração 0
x =  [0.7, -1.6, 0.6]
Iteração:  1
xk =  [0.96, -1.8600000000000003, 0.9400000000000001]
Iteração:  2
xk =  [0.9780000000000002, -1.9800000000000002, 0.9660000000000001]
Iteração:  3
xk =  [0.9994000000000001, -1.9888000000000003, 0.9984000000000001]
Iteração:  4
xk =  [0.9979200000000001, -1.99956, 0.9967600000000001]
Iteração:  5
xk =  [1.000236, -1.9989360000000003, 1.000284]
x =  [1.000236, -1.9989360000000003, 1.000284]
